In [3]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from requests.exceptions import HTTPError
import sys
import math

## Reading floorsheet file


In [18]:

df = pd.read_csv('./datas/floorsheet_2021-06-24_Thursday.csv', low_memory=False)
df.head(20)
df.drop_duplicates()

,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount,Time
0,1,HPPL,54,56,100,408.0,40800.0,14:59:59
1,2,EDBL,56,32,10,787.0,7870.0,14:59:59
2,3,EDBL,56,58,70,787.0,55090.0,14:59:59
3,4,NLICL,19,14,100,1174.0,117400.0,14:59:59
4,5,NHPC,38,35,10,323.0,3230.0,14:59:59
...,...,...,...,...,...,...,...,...
78978,78979,DDBL,42,16,10,1598.0,15980.0,10:46:00
78979,78980,NHPC,1,39,98,335.0,32830.0,10:46:00
78980,78981,NHPC,1,39,300,335.0,100500.0,10:46:00
78981,78982,NHPC,1,20,2,335.0,670.0,10:46:00


## Filtering

In [16]:
c1 = df["Amount"] > 1000000
c2 = df["Symbol"] == "GUFL"
c3 = df["Buyer"] == "58"
condition = c1&c3 ##& is and operator
df[(condition)].sort_values("Amount", ascending =False)

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
19382,19382,19383,SHPC,58,49,21081,490.0,10329690.0
18813,18813,18814,ALICL,58,44,3729,1362.0,5078898.0
61315,61315,61316,NICA,58,4,5000,893.0,4465000.0
17031,17031,17032,SHPC,58,49,8919,490.0,4370310.0
26371,26371,26372,NICA,58,42,4746,897.0,4257162.0
...,...,...,...,...,...,...,...,...
60238,60238,60239,MFIL,58,37,1000,1015.0,1015000.0
60240,60240,60241,MFIL,58,55,1000,1014.0,1014000.0
60239,60239,60240,MFIL,58,55,1000,1014.0,1014000.0
66023,66023,66024,MFIL,58,55,1000,1014.0,1014000.0


## Buyer broker analysis

In [8]:
c3 = df["Buyer"]== "58"
df[c3].groupby([ "Buyer", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Buyer  Symbol
58     API       13186086.0
       CHDC       6810380.0
       ALICL      5805602.0
       AKPL       3155089.0
       BOKL       2944325.0
       AHPC       2900420.0
       CCBL       2595993.0
       BPCL       2518871.0
       CBL        2327212.0
       BFC        2137301.0
       CBBL       1693277.0
       AIL        1174420.0
       CFCL       1158960.0
       CGH        1082910.0
       CHCL        748522.0
       BARUN       617590.0
       ADBL        599542.0
       AKJCL       451197.0
       ACLBSL      129030.0
       ALBSL        57640.0
Name: Amount, dtype: float64

## Seller broker analysis

In [12]:
c = df["Seller"] == "58"
df[c].groupby([ "Seller", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Seller  Symbol
58      ADBL      4471593.0
        AKPL      4287417.0
        AHPC      3796797.0
        BPCL      2084964.0
        ALICL     1883832.0
        API       1471804.0
        CFCL       854247.0
        CBL        764595.0
        CBBL       757920.0
        CHCL       549458.0
        BOKL       523382.0
        CCBL       393939.0
        CGH        375820.0
        BBC        292850.0
        BARUN      251402.0
        AIL        206900.0
        BFC        147640.0
        ALBSL      137273.0
        ACLBSL      78758.0
        AKJCL       61443.0
Name: Amount, dtype: float64

## Stockwise top buy 

In [5]:
c = df["Symbol"] == "GUFL"
df[c].groupby([ "Symbol", "Buyer"])["Amount"].sum().head(20).sort_values(ascending=False)

Symbol  Buyer
GUFL    20       36769427.0
        3        11496746.0
        44        6217845.0
        28        4615009.0
        35        3484102.0
        36        1303450.0
        14        1059092.0
        17         985070.0
        38         873290.0
        34         727700.0
        19         685300.0
        37         566500.0
        43         452361.0
        33         384784.0
        16         287500.0
        21         246900.0
        22         237150.0
        42         164145.0
        41          67617.0
        11          45240.0
Name: Amount, dtype: float64

## Stockwise top sell

In [17]:
c = df["Symbol"] == "GUFL"
df[c].groupby([ "Symbol", "Seller"])["Amount"].sum().head(20).sort_values(ascending=False)

Symbol  Seller
GUFL    3         13363935.0
        17         5305358.0
        28         2755903.0
        39         1689250.0
        40         1521004.0
        29         1505859.0
        35         1165000.0
        38          836320.0
        14          773400.0
        22          653130.0
        34          588970.0
        10          539100.0
        16          259420.0
        26          251560.0
        20          116900.0
        36           89346.0
        18           58450.0
        32           57700.0
        25           22420.0
        4            11650.0
Name: Amount, dtype: float64

## Companywise turnover

In [13]:
company = df["Symbol"] == "GUFL"
df[(company)].groupby(["Symbol"])["Amount"].sum().sort_values(ascending = False).head(20)

Symbol
GUFL    90542388.0
Name: Amount, dtype: float64